# Image-Classification-For-Cross-Analysis-of-Chest-X-Ray

In [1]:
# Install any necessary environment if code presents errors such as downgrading:
!pip install tensorflow==2.14.1
# trying to match environment the keras file was saved in

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling te

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.9 MB/s e

In [3]:
"""
Loads a pretrained CNN model and presents a Gradio app to the user.
The user can upload a chest X-Ray image which will be preprocessed similarly
to the trained model, then run through the model.
A prediction of 1 or more of 14 diagnoses or no diagnosis will be output.
This will be followed up with suggested treatment from an OpenAI API.
"""

# Standard imports
import pandas as pd
from pathlib import Path
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
from pathlib import Path
import gradio as gr

In [4]:
# Connect to my Google Drive to get to Pickle File
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
# first lets load our pre-trained model
xray_model = tf.keras.models.load_model(Path("/content/drive/My Drive/xrays.keras"))

In [58]:
# Define the function of our model that will process the image and meta data to make a prediction
def do_predict(age, gender, view, img):
    # Evem though specified image as PIL.Image it's still converting so convert back
    img = Image.fromarray(img)
    # Preprocess image
    if img.mode != "L":
        # Image is not 8 bit so convert it
        img = img.convert(mode="L", dither=None, colors=256)
    # Resize image (based on my model, can change for other models)
    target_size = (256, 256)
    img = img.resize(target_size, resample = Image.LANCZOS)
    # Convert to float and normalize between 0 and 1
    img = np.array(img).astype(np.float32)
    img = img/255
    # Add channel dimension and the batch size dimension
    X_img = np.expand_dims(img, axis=-1)
    X_img = np.expand_dims(X_img, axis=0)

    # Oops, just realized that I scaled the meta-data;
    #   should have only scaled the age and pickled its scaler instance for this.
    #   As my model won't be used, just testing Gradio interface here, will just estimate the scale
    age = age/85
    # combine meta-data into dataframe as initial modeling and convert to numpy
    X_cat = pd.DataFrame({"Patient Age" : [age], "Patient Gender" : [gender], "View Position" : [view]})
    X_cat = np.array(X_cat)

    prediction = xray_model.predict([X_cat, X_img])

    # Proof of concept on this file completed, the chosen model will inform the
    # actual inner workings of this fuction which should output both
    # the diagnosis prediction (any positive values), and an OpenAI suggested treatment
    # into the output box.
    return f"Prediction: {prediction}"

In [59]:
# Create an instance of the Gradio Interface application function with parameters.
app = gr.Interface(fn=do_predict,
                   title="Chest X-Ray Cross Analysis",
                   inputs=[gr.Number(label="Age", minimum=0, maximum=100),
                           gr.Dropdown(label="Gender", choices=["Female", "Male"], type="index"),
                           gr.Dropdown(label="View Position", choices=["AP", "PA"], type="index"),
                           gr.Image(value="PIL.Image.Image")],
                   outputs=gr.Textbox(lines=20, label="Prediction", show_copy_button=True))
# Launch the app
app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c289b5b7ec14e3feb0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
